# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Santiago Gaviria Zapata
#### Segundo Integrante: Jorge Ricardo Hiler

In [2]:
from __future__ import division
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [78]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [4]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor

#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [79]:
from numpy import random
from numpy import matlib as npmatlib
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
#Validamos el modelo

def train(neurons, layers):
    Folds = 4
    random.seed(19680801)
    ErrorY1 = np.zeros(Folds)
    ErrorY2 = np.zeros(Folds)
    mlp = np.zeros(Folds)
    ErrorT = np.zeros(Folds)
    ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
    j = 0
    for train, test in ss.split(X):
        print(X.shape)
        print(Y.shape)
        Xtrain = X[train,:]
        Ytrain = Y[train,:]
        Xtest = X[test,:]
        Ytest = Y[test,:]

        #Normalizamos los datos
        media = np.mean(Xtrain,axis=0)
        desvia = np.std(Xtrain,axis=0)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - npmatlib.repmat(media, Xtest.shape[0], 1))/npmatlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if(neurons == 1):
            mlp = MLPRegressor(hidden_layer_sizes=(layers,), activation='tanh', solver='adam', max_iter=500)
        elif(neurons == 2):
            mlp = MLPRegressor(hidden_layer_sizes=(layers,neurons), activation='tanh', solver='adam', max_iter=500)
        mlp = mlp.fit(Xtrain, Ytrain)
        #Use para el modelo para hacer predicciones sobre el conjunto Xtest
        Yest = mlp.predict(Xtest)

        #Mida el error MAPE para cada una de las dos salida
        ErrorY1[j] = MAPE(Yest[:,0], Ytest[:,0])
        ErrorY2[j] = MAPE(Yest[:,1], Ytest[:,1])
        ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
        j += 1
    return np.mean(ErrorY1), np.std(ErrorY1), np.mean(ErrorY2), np.std(ErrorY2)


def fillTable(neurons, layers):
    i = 0
    MAPESalida1 = np.zeros(10)
    ICSalida1 = np.zeros(10) 
    MAPESalida2 = np.zeros(10) 
    ICSalida2 = np.zeros(10)
    for neuron, layer in zip(neurons, layers):
        MAPESalida1[i], ICSalida1[i], MAPESalida2[i], ICSalida2[i] = train(neuron, layer)
        i += 1
    return MAPESalida1, ICSalida1, MAPESalida2, ICSalida2

(9357, 11)
(9357, 2)


## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [68]:
import pandas as pd
import qgrid
MAPESalida1, ICSalida1, MAPESalida2, ICSalida2 = fillTable([1,1,1,1,1,2,2,2,2,2],[20,24,28,32,36,20,24,28,32,36])
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = MAPESalida1
df_types["IC MAPE salida 1"] = ICSalida1
df_types["MAPE salida 2"] = MAPESalida2
df_types["IC MAPE salida 2"] = ICSalida2
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [69]:
qgrid_widget.get_changed_df()

MAPE salida 1  IC MAPE salida 1  \
N. de capas ocultas Neuronas por capa                                    
1                   20                      0.086230          0.001955   
                    24                      0.081843          0.001867   
                    28                      0.077248          0.001354   
                    32                      0.076859          0.001732   
                    36                      0.075092          0.000897   
2                   20                      0.315003          0.061116   
                    24                      0.346548          0.058340   
                    28                      0.366723          0.032001   
                    32                      0.319796          0.064391   
                    36                      0.382938          0.008221   

                                       MAPE salida 2  IC MAPE salida 2  
N. de capas ocultas Neuronas por capa                                   
1                   20                      2.398581          0.158548  
                    24                      2.061491          0.109601  
                    28                      1.732973          0.069428  
                    32                      1.516950          0.050861  
                    36                      1.501080          0.038991  
2                   20                      6.008079          0.343087  
                    24                      6.085216          0.493971  
                    28                      6.214105          0.353557  
                    32                      6.075492          0.485815  
                    36                      5.814684          0.312095

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

La función de activación que usa el modelo en la capa de salida es la de tangente inversa.

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [80]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [97]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

def train(neurons, layers):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if(neurons == 1):
            mlp = MLPClassifier(hidden_layer_sizes=(layers,), activation='tanh', solver='adam', max_iter=500)
        elif(neurons == 2):
            mlp = MLPClassifier(hidden_layer_sizes=(layers,neurons), activation='tanh', solver='adam', max_iter=500)
        mlp = mlp.fit(Xtrain, Ytrain)

        #Validación con las muestras de entrenamiento
        Ytrain_pred = mlp.predict(Xtrain)

        #Validación con las muestras de test
        Yest = mlp.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1

    return np.mean(EficienciaVal), np.std(EficienciaVal)
    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    
def fillTable(neurons, layers, length):
    i = 0
    EficienciaVal = np.zeros(length)
    ICVal = np.zeros(length) 
    for neuron, layer in zip(neurons, layers):
        EficienciaVal[i], ICVal[i] = train(neuron, layer)
        i += 1
    return EficienciaVal, ICVal

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9623501551552112+-0.033708835273301894


## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [96]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
EficienciaVal, ICVal = fillTable([1,1,1,1,1,2,2,2,2,2], [20,24,28,32,36,20,24,28,32,36], 10)
df_types["Eficiencia en validacion"] = EficienciaVal
df_types["Intervalo de confianza"] = ICVal
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/sady/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  %

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [98]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  \
N. de capas ocultas Neuronas por capa                             
1                   20                                 0.956794   
                    24                                 0.962350   
                    28                                 0.963785   
                    32                                 0.965175   
                    36                                 0.965120   
2                   20                                 0.868299   
                    24                                 0.924978   
                    28                                 0.931961   
                    32                                 0.835457   
                    36                                 0.943035   

                                       Intervalo de confianza  
N. de capas ocultas Neuronas por capa                          
1                   20                               0.026823  
                    24                               0.033709  
                    28                               0.030349  
                    32                               0.024554  
                    36                               0.031155  
2                   20                               0.046806  
                    24                               0.034814  
                    28                               0.030125  
                    32                               0.101638  
                    36                               0.030272

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 



4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?